# El Viejo XOR: Métodos No-lineales para Problemas No-lineales

In [ ]:
explicacion

## A)

In [ ]:
import numpy as np

def do_XOR(n=1000,noisy_n=100,svm=True):
    rng = np.random.RandomState(0)
    X_train = rng.uniform(low=-1.0,high=1.0,size=(n,2))
    Y_train = np.logical_xor(X_train[:,0] > 0, X_train[:,1] > 0)
    Y_train = 2*Y_train-1 if svm else Y_train
    X_noisy= rng.uniform(low=-0.8,high=0.2,size=(noisy_n,2))
    Y_noisy = -1*np.logical_xor(X_noisy[:,0] > 0, X_noisy[:,1] > 0) + 1
    Y_noisy = 2*Y_noisy-1 if svm else Y_noisy
    X_train = np.vstack((X_train, X_noisy))
    Y_train = np.hstack((Y_train, Y_noisy))
    X_test = rng.uniform(low=-1.0,high=1.0,size=(n,2))
    Y_test = np.logical_xor(X_test[:,0] > 0, X_test[:,1] > 0)
    Y_test = 2*Y_test - 1 if svm else Y_test
    return X_train,Y_train,X_test,Y_test

X_train,Y_train,X_test,Y_test = do_XOR()

## B)

In [ ]:
import matplotlib.pyplot as plt

def plot_classifier(clf,X_train,Y_train,X_test,Y_test,model_type):
    f, axis = plt.subplots(1, 1, sharex=’col’, sharey=’row’,figsize=(20, 20))
    axis.scatter(X_train[:,0],X_train[:,1],s=5,c=Y_train,zorder=10,cmap=’gray’)
    axis.scatter(X_test[:,0],X_test[:,1],s=15,c=Y_test,zorder=10,cmap=’gray’)
    XX, YY = np.mgrid[-1:1:200j, -1:1:200j]
    if model_type == ’svm’:
    Z = clf.decision_function(np.c_[XX.ravel(), YY.ravel()])
    elif model_type == ’tree’:
    Z = clf.predict_proba(np.c_[XX.ravel(), YY.ravel()])[:,0]
    elif model_type == ’ann’:
    Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])
    else: raise ValueError(’model type not supported’)
    Z = Z.reshape(XX.shape)
    Zplot = Z > 0 if model_type == ’svm’ else Z > 0.5
    axis.pcolormesh(XX, YY, Zplot ,cmap=’YlGn’)
    axis.contour(XX, YY, Z, alpha=1, colors=[’k’, ’k’, ’k’],
    linestyles=[’--’, ’-’, ’--’],levels=[-1, 0, 1])
    plt.show()

## C)

In [ ]:
from sklearn.svm import SVC
clf = SVC(C=100, kernel=’rbf’)
clf = SVC(C=10, kernel=’poly’,degree=2, coef0=1)
clf.fit(X_train, Y_train)
print "Test Accuracy = %f"%clf.score(X_test,Y_test)
plot_classifier(clf,X_train,Y_train,X_test,Y_test,’svm’)

## D)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
n_h=1
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], init=’uniform’, activation=’relu’))
model.add(Dense(n_h, init=’uniform’, activation=’sigmoid’))
model.compile(optimizer=SGD(lr=1), loss=’binary_crossentropy’, metrics=[’accuracy’])
model.fit(X_train, Y_train, nb_epoch=50, batch_size=100, verbose=1)
scores = model.evaluate(X_test, Y_test)
test_acc = scores[1]

## E)

In [ ]:
# Hacer algo

n_h=8
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], init=’uniform’, activation=’relu’))
model.add(Dense(n_h, init=’uniform’, activation=’sigmoid’))

## F)

In [ ]:
from sklearn.tree import DecisionTreeClassifier as Tree
clf=Tree(criterion=’gini’,splitter=’best’,random_state=0,max_depth=1)
clf.fit(X_train,Y_train)
acc_test = clf.score(X_test,Y_test)
print "Test Accuracy = %f"%acc_test
print clf.tree_.max_depth
plot_classifier(clf,X_train,Y_train,X_test,Y_test,’tree’)

## G)

In [ ]:
# Hacer algo

n_t=8
clf=Tree(criterion=’gini’,splitter=’best’,random_state=0,max_depth=n_t)
clf.fit(X_train,Y_train)